In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

## Preprocessing 

In [2]:
df = pd.read_excel('data.xlsx', engine='openpyxl')
df.head(20)

,ctx,codice_dm,lista_id,pers_sesso,pers_eta_ingresso_in_lista,diagnosi,diagnosi_specifica,eltr,lista_ingresso_data,tot_tx_organo,...,donatore_HBeAg,donatore_HBeAb,donatore_antiCoreTot,donatore_hbv_dna,donatore_hcv_rna,donatore_steatosiMacro,donatore_biopsiaEpatica,donatore_biopsiaRenale,donatore_livelloRischioPreLT,donatore_variazioneLivello
0,BOLOGNA,12898,20412,M,54,Altra epatopatia,NaN,NaN,2004-03-10,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BOLOGNA,12908,20422,M,64,Cirrosi postnecrotica da HCV,NaN,NaN,2003-12-02,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BOLOGNA,12909,20423,M,56,Cirrosi criptogenetica o idiopatica,NaN,D11 : Cirrhosis-Cryptogenic (unknown) cirrhosis,2002-07-18,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BOLOGNA,12913,20428,M,51,Cirrosi di Laennec (post-alcolica),NaN,NaN,2003-06-26,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BOLOGNA,12935,20450,M,43,Cirrosi postnecrotica da HCV,NaN,NaN,2004-01-08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,BOLOGNA,12937,20452,F,63,Epatocarcinoma associato a cirrosi,NaN,NaN,2003-12-29,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,BOLOGNA,12939,20454,F,55,Cirrosi biliare primitiva,NaN,NaN,2003-10-02,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,BOLOGNA,12958,20473,M,63,Cirrosi postnecrotica da HCV,NaN,NaN,2003-07-31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,BOLOGNA,12961,20476,M,62,Cirrosi postnecrotica da HBV HbsAg positiva,NaN,NaN,2003-07-24,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BOLOGNA,12964,20479,M,43,Cirrosi postnecrotica da HCV,NaN,NaN,2004-02-23,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df['diagnosi'].value_counts()

Epatocarcinoma associato a cirrosi                                                              708
Cirrosi postnecrotica da HCV                                                                    425
Cirrosi di Laennec (post-alcolica)                                                              249
Ritrapianto/rigetto di trapianto epatico                                                         90
Cirrosi postnecrotica da HBV HbsAg positiva                                                      72
Cirrosi postnecrotica da coinfezione HBV + HDV                                                   67
Cirrosi criptogenetica o idiopatica                                                              64
Altra epatopatia                                                                                 58
Cirrosi di Laennec associata a cirrosi postnecrotica                                             52
Cirrosi biliare primitiva                                                                        48


In [4]:
missing_percent = {}
for column in df.columns:
    missing_percent[column] = df[column].isna().sum() / len(df)
for column in sorted(missing_percent, key=missing_percent.get, reverse=True):
    print('{}: {:.3f}'.format(column, missing_percent[column]))

Na: 1.000
fega_note: 0.999
donatore_diabete_insulinodipendente_desc: 0.999
donatore_bassaportata3_PAmin: 0.998
donatore_bassaportata3_data: 0.997
donatore_bassaportata3_ora: 0.997
donatore_bassaportata3_durata: 0.997
donatore_bassaportata3_PAmax: 0.997
donatore_bassaportata3_trattamento: 0.997
ISO2_tipo_super_urgenza: 0.997
donatore_bassaportata2_PAmin: 0.995
donatore_bassaportata2_PAmax: 0.993
donatore_bassaportata2_durata: 0.991
donatore_bassaportata2_ora: 0.990
donatore_bassaportata2_trattamento: 0.990
scheda48h_tipo_tx_effettivo_combinato: 0.989
donatore_biopsiaRenale: 0.989
donatore_bassaportata2_data: 0.988
donatore_note: 0.980
categoria_HCC: 0.977
diagnosi_specifica: 0.974
donatore_diabete_desc: 0.974
donatore_dislipidemie_desc: 0.970
trombosi_portale_note: 0.969
ISO2_Ecc. MELD: 0.969
donatore_bassaportata1_PAmin: 0.958
donatore_variazioneLivello: 0.954
tipo_combinato: 0.944
ric_eccezione_MELD_PELD: 0.944
donatore_bassaportata1_PAmax: 0.939
donatore_HBeAg: 0.939
donatore_bassapo

In [5]:
columns = ['BMI', 'MELD_base',
           'pers_sesso', 'ric_diabete',
           'ric_HCV', 'data_trapianto',
           'data_decesso', 'data_ultimo_agg_followup',
           'pers_eta_ingresso_in_lista', 'HCC',
           'trombosi_portale', 'ric_HBsAg',
           'UNOS', 'ric_HIV', 'lista_ingresso_data',
           'chirurgia_addom', 'iscriz_tx_comb', 
           'lista_uscita_data', 'causa_uscita', 
           'MELD_in_ingresso', 'MELD_in_uscita']

categories = ['pers_sesso', 'ric_diabete',
              'ric_HCV', 'HCC', 'trombosi_portale',
              'ric_HBsAg', 'UNOS', 'ric_HIV',
              'chirurgia_addom', 'iscriz_tx_comb']

# column that need to be explained: CHILD, UNOS, diagnosi, codice dm, causa_uscita

In [6]:
df = df[columns]
df = pd.get_dummies(df, columns=categories, drop_first=True)
df['deceased_censored'] = np.zeros(len(df))
df['transplanted'] = np.zeros(len(df))
df['ultimo_aggiornamento'] = df['data_decesso']

df.head(50)
# how do we handle patients in the list that do not have data_trapianto and data_decesso
# what does 'Non idonea clinicamente' mean?

,BMI,MELD_base,data_trapianto,data_decesso,data_ultimo_agg_followup,pers_eta_ingresso_in_lista,lista_ingresso_data,lista_uscita_data,causa_uscita,MELD_in_ingresso,...,ric_HBsAg_SI,UNOS_2A,UNOS_2B,UNOS_3,ric_HIV_SI,chirurgia_addom_SI,iscriz_tx_comb_SI,deceased_censored,transplanted,ultimo_aggiornamento
0,25.16,8.0,2004-10-22,2013-08-01,NaT,54,2004-03-10,2004-10-22,TX nel centro di iscrizione,NaN,...,0,0,1,0,0,0,1,0.0,0.0,2013-08-01
1,27.76,10.0,NaT,NaT,NaT,64,2003-12-02,2010-01-12,Non idoneo clinicamente,NaN,...,0,0,1,0,0,1,0,0.0,0.0,NaT
2,26.49,13.0,NaT,NaT,NaT,56,2002-07-18,2016-06-09,Uscito per miglioramento,NaN,...,0,0,1,0,0,0,0,0.0,0.0,NaT
3,25.54,14.0,NaT,2004-01-28,NaT,51,2003-06-26,2004-01-28,Deceduto iscritto in lista,NaN,...,0,0,1,0,0,0,0,0.0,0.0,2004-01-28
4,21.47,16.0,NaT,2005-06-11,NaT,43,2004-01-08,2005-06-11,Deceduto iscritto in lista,NaN,...,0,0,0,1,0,0,0,0.0,0.0,2005-06-11
5,22.15,19.0,2004-12-01,2006-05-15,NaT,63,2003-12-29,2004-12-01,TX nel centro di iscrizione,NaN,...,0,0,1,0,0,1,0,0.0,0.0,2006-05-15
6,18.36,13.0,NaT,2004-08-25,NaT,55,2003-10-02,2004-08-25,Deceduto iscritto in lista,NaN,...,0,0,1,0,0,1,0,0.0,0.0,2004-08-25
7,27.94,21.0,NaT,2005-02-18,NaT,63,2003-07-31,2005-02-18,Deceduto iscritto in lista,NaN,...,0,0,1,0,0,0,0,0.0,0.0,2005-02-18
8,24.57,16.0,NaT,NaT,NaT,62,2003-07-24,2006-09-21,Non idoneo clinicamente,NaN,...,1,0,0,0,0,1,0,0.0,0.0,NaT
9,NaN,18.0,2004-03-22,2004-03-22,NaT,43,2004-02-23,2004-03-22,TX nel centro di iscrizione,NaN,...,0,0,1,0,0,0,0,0.0,0.0,2004-03-22


In [7]:
#for index, row in df.iterrows():
#    if 

#for index, row in df.iterrows():
#    if row['data_ultimo_agg_followup'] is pd.NaT and not row['data_decesso'] is pd.NaT:
#        df.at[index,'data_ultimo_agg_followup'] = row['data_decesso']
#        df.at[index, 'deceased_censored'] = 1
    #elif (row['data_decesso'] is pd.NaT and not row['data_ultimo_agg_followup'] is pd.NaT:
    #    df.at[index,'data_decesso'] = row['data_ultimo_agg_followup']
        
#df['ToS'] = (df['data_ultimo_agg_followup'] - df['data_trapianto']).dt.days
#df = df.reset_index()
#df = df.drop(['data_decesso', 'data_trapianto', 'data_ultimo_agg_followup', 'index'], axis=1)

In [8]:
#df['causa_uscita'].value_counts()